In [1]:
import pandas as pd
import os

dataset_rename = {'cw': 'ContentWise', 'rl4rs': 'RL4RS'}
metrics_rename = {'f1':'F1', 'accuracy': 'Accuracy', 'roc-auc':"ROC-AUC"}
embeddings_rename = {'svd':'SVD', 'neural':"nn.Embeddings", 'explicit':"Explicit"}

files = [file for file in os.listdir('.') if file.endswith('csv') and file.split('_')[0] in dataset_rename]
model_names = list(set([file.split('_')[-1].split('.')[0] for file in files]))
print('== all models ==', *model_names, sep='\n')

# slatewise
slatewise_models = {
    'MatrixFactorization': 'MF', 
    'LogReg': 'Logistic regression', 
    'SlatewiseGRU': 'Slate-wise GRU', 
    # 'NeuralClickModelBest': 'NeuralClickModel',
    'SlatewiseAttention' : 'Slate-wise Transformer'
}

# sessionwise
sessionwise_models = {
    'SCOT': 'SCOT', 
    'AttentionGRU' : 'Transformer + GRU', 
    'SessionwiseGRU': 'Session-wise GRU',
    'SessionwiseAttention': 'Session-wise Transformer'
}

model_names = slatewise_models
model_names.update(sessionwise_models)

== all models ==
SlatewiseAttention
SlatewiseGRU
SessionwiseGRU
SCOT2
NeuralClickModelBase
MatrixFactorization
NeuralClickModel
NeuralClickModelReadoutTestOnly
LogReg
SCOT
SessionwiseAttention
NeuralClickModelBest
AttentionGRU


In [2]:
data = {}

index = pd.MultiIndex.from_product(
    [["ContentWise", "RL4RS"], ["ROC-AUC", "F1", "Accuracy"]]
)

columns = pd.MultiIndex.from_product(
    [model_names.values(), ["SVD", "nn.Embeddings", "Explicit"]]
)


df = pd.DataFrame(
    index=index,
    columns=columns,
)
data = df.to_dict()
data

{('MF', 'SVD'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS', 'Accuracy'): nan},
 ('MF', 'nn.Embeddings'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS', 'Accuracy'): nan},
 ('MF', 'Explicit'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS', 'Accuracy'): nan},
 ('Logistic regression', 'SVD'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS', 'Accuracy'): nan},
 ('Logistic regression', 'nn.Embeddings'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): n

In [3]:
for file in files:
    dataset = dataset_rename[file.split('_')[0]]
    try:
        model = model_names[file.split('_')[-1].split('.')[0]]
    except:
        print(f'Warning: {file} is unknown model')
        continue
    data_dict = pd.read_csv(file).drop('Unnamed: 0', axis=1).set_index('embeddings').to_dict('index')
    for embedding, m in data_dict.items():
        for metric, score in m.items():
#             print(model, embeddings_rename[embedding], dataset, metrics_rename[metric], score )
            try:
                data[model, embeddings_rename[embedding]][dataset, metrics_rename[metric]] = score
            except:
                pass
        

In [4]:
df = pd.DataFrame(
    data,
    index=index,
    columns=columns,
).T.dropna(how='all')
df

ContentWise                      \
                                           ROC-AUC        F1  Accuracy   
MF                       SVD              0.650284  0.200673  0.261733   
Logistic regression      SVD              0.625023  0.245934  0.784351   
                         nn.Embeddings    0.689394  0.269616  0.786626   
                         Explicit              NaN       NaN       NaN   
Slate-wise GRU           SVD              0.682557  0.279316  0.832012   
                         nn.Embeddings    0.728361  0.280054  0.888137   
                         Explicit              NaN       NaN       NaN   
Slate-wise Transformer   SVD              0.671754  0.269316  0.869375   
                         nn.Embeddings    0.745903  0.312001  0.870368   
                         Explicit              NaN       NaN       NaN   
SCOT                     SVD              0.671757  0.297491  0.863569   
                         nn.Embeddings    0.715095  0.306451  0.812614   
                         Explicit              NaN       NaN       NaN   
Transformer + GRU        SVD              0.688890  0.286763  0.861957   
                         nn.Embeddings    0.737799  0.309336  0.863254   
                         Explicit              NaN       NaN       NaN   
Session-wise GRU         SVD              0.668916  0.252537  0.864762   
                         nn.Embeddings    0.722389  0.302965  0.805752   
                         Explicit              NaN       NaN       NaN   
Session-wise Transformer SVD              0.637128  0.261067  0.817086   
                         nn.Embeddings    0.703817  0.284120  0.773755   
                         Explicit              NaN       NaN       NaN   

                                           RL4RS                      
                                         ROC-AUC        F1  Accuracy  
MF                       SVD            0.719054  0.761109  0.657841  
Logistic regression      SVD            0.863317  0.846207  0.789941  
                         nn.Embeddings  0.914967  0.880251  0.837239  
                         Explicit       0.863242  0.843943  0.769000  
Slate-wise GRU           SVD            0.907823  0.867172  0.834022  
                         nn.Embeddings  0.919320  0.879220  0.847225  
                         Explicit       0.915392  0.880290  0.845968  
Slate-wise Transformer   SVD            0.898504  0.866035  0.824713  
                         nn.Embeddings  0.932605  0.890147  0.848942  
                         Explicit       0.865922  0.843821  0.766655  
SCOT                     SVD            0.909597  0.874627  0.831266  
                         nn.Embeddings  0.933914  0.895775  0.865482  
                         Explicit       0.927273  0.891119  0.859896  
Transformer + GRU        SVD            0.917050  0.871182  0.838786  
                         nn.Embeddings  0.933702  0.892773  0.859799  
                         Explicit       0.878918  0.856078  0.802080  
Session-wise GRU         SVD            0.921209  0.885263  0.847975  
                         nn.Embeddings  0.923012  0.888139  0.847685  
                         Explicit       0.923664  0.889565  0.853440  
Session-wise Transformer SVD            0.880952  0.848669  0.781864  
                         nn.Embeddings  0.925115  0.885014  0.842776  
                         Explicit       0.913644  0.876479  0.831411

In [5]:
df = df.style.format(decimal='.', precision=3).highlight_max(axis=0, props="font-weight:bold;")

In [6]:
df

In [7]:
print(df.to_latex(convert_css=True))

\begin{tabular}{llrrrrrr}
 &  & \multicolumn{3}{r}{ContentWise} & \multicolumn{3}{r}{RL4RS} \\
 &  & ROC-AUC & F1 & Accuracy & ROC-AUC & F1 & Accuracy \\
MF & SVD & 0.650 & 0.201 & 0.262 & 0.719 & 0.761 & 0.658 \\
\multirow[c]{3}{*}{Logistic regression} & SVD & 0.625 & 0.246 & 0.784 & 0.863 & 0.846 & 0.790 \\
 & nn.Embeddings & 0.689 & 0.270 & 0.787 & 0.915 & 0.880 & 0.837 \\
 & Explicit & nan & nan & nan & 0.863 & 0.844 & 0.769 \\
\multirow[c]{3}{*}{Slate-wise GRU} & SVD & 0.683 & 0.279 & 0.832 & 0.908 & 0.867 & 0.834 \\
 & nn.Embeddings & 0.728 & 0.280 & \bfseries 0.888 & 0.919 & 0.879 & 0.847 \\
 & Explicit & nan & nan & nan & 0.915 & 0.880 & 0.846 \\
\multirow[c]{3}{*}{Slate-wise Transformer} & SVD & 0.672 & 0.269 & 0.869 & 0.899 & 0.866 & 0.825 \\
 & nn.Embeddings & \bfseries 0.746 & \bfseries 0.312 & 0.870 & 0.933 & 0.890 & 0.849 \\
 & Explicit & nan & nan & nan & 0.866 & 0.844 & 0.767 \\
\multirow[c]{3}{*}{SCOT} & SVD & 0.672 & 0.297 & 0.864 & 0.910 & 0.875 & 0.831 \\
 & nn.Embe